## 用向前法建立子集回归模型

### 源代码如下：

In [1]:
library(xlsx)
df<-read.xlsx("5-7.xlsx",1)[,1:6]

In [2]:
lm.0<-lm(y~1,data=df)
# summary(lm.0)
add1(lm.0,scope=.~.+x1+x2+x3+x4+x5,test="F")

,Df,Sum of Sq,RSS,AIC,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
<none>,NA,NA,14681.928,182.5852,NA,NA
x1,1,5783.7802,8898.148,170.0628,17.5499512,2.677828e-04
x2,1,153.8070,14528.121,184.2798,0.2858449,5.972713e-01
x3,1,184.4707,14497.458,184.2185,0.3435574,5.626514e-01
x4,1,10578.6846,4103.244,147.6149,69.6094369,5.935009e-09
x5,1,1784.3647,12897.564,180.8274,3.7354222,6.382273e-02


In [3]:
lm.1<- update(lm.0,formula=.~.+x4)
# summary(lm.1)

In [4]:
add1(lm.1,scope=.~.+x1+x2+x3+x4+x5,test="F")

,Df,Sum of Sq,RSS,AIC,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
<none>,NA,NA,4103.244,147.6149,NA,NA
x1,1,195.8051638,3907.439,148.1969,1.302883e+00,2.640916e-01
x2,1,0.1154703,4103.128,149.6141,7.316922e-04,9.786266e-01
x3,1,2028.9136301,2074.330,129.8328,2.543074e+01,3.001027e-05
x5,1,1473.8844079,2629.359,136.7088,1.457427e+01,7.508875e-04


In [5]:
lm.2<- update(lm.1,formula=.~.+x3)
# summary(lm.2)

In [6]:
add1(lm.2,scope=.~.+x1+x2+x3+x4+x5,test="F")

,Df,Sum of Sq,RSS,AIC,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
<none>,NA,NA,2074.330,129.8328,NA,NA
x1,1,89.17209,1985.158,130.5586,1.1229848,0.29940822
x2,1,226.26853,1848.062,128.4833,3.0608900,0.09246551
x5,1,77.80531,1996.525,130.7242,0.9742592,0.33307753


### 统计分析：

每步在未进入模型的变量中挑选一个加入模型并建立回归模型，对新加入的变量进行F检验；对每一个变量都这样做，得到一系列F检验值。如果有变量通过了显著性检验，则在通过了显著性检验的变量中挑选F值最大的变量加入模型；否则，终止算法。

上面各表的最后一列是p值，p值小于0.05代表该变量是显著的。如上各表：第一个表中变量$x_1,x_4$通过了显著性检验，且$x_4$的F值最大，于是引入$x_4$。第二张表中变量$x_3,x_5$通过了显著性检验，且$x_3$的F值最大，于是引入$x_3$。第三张表中没有变量通过显著性检验，则终止算法。



## 用逐步回归法建立子集回归模型

In [27]:
library("caret")

In [28]:
library(caret)
step_model<-train(y~.,
                  data = df,
                  method = "leapSeq",
                  tuneGrid = data.frame(nvmax = 1:5),#变量个数
                #   trControl = ctrl
                  )
summary(step_model)

Subset selection object
5 Variables  (and intercept)
   Forced in Forced out
x1     FALSE      FALSE
x2     FALSE      FALSE
x3     FALSE      FALSE
x4     FALSE      FALSE
x5     FALSE      FALSE
1 subsets of each size up to 2
Selection Algorithm: 'sequential replacement'
         x1  x2  x3  x4  x5 
1  ( 1 ) " " " " " " "*" " "
2  ( 1 ) " " " " "*" "*" " "

In [29]:
step_model$bestTune

,nvmax
,<int>
2,2


In [10]:
lm.step<- step(lm.0,scope=.~.+x1+x2+x3+x4+x5,test="F",direction = "both")

Start:  AIC=182.59
y ~ 1

       Df Sum of Sq     RSS    AIC F value    Pr(>F)    
+ x4    1   10578.7  4103.2 147.62 69.6094 5.935e-09 ***
+ x1    1    5783.8  8898.1 170.06 17.5500 0.0002678 ***
+ x5    1    1784.4 12897.6 180.83  3.7354 0.0638227 .  
<none>              14681.9 182.59                      
+ x3    1     184.5 14497.5 184.22  0.3436 0.5626514    
+ x2    1     153.8 14528.1 184.28  0.2858 0.5972713    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Step:  AIC=147.61
y ~ x4

       Df Sum of Sq     RSS    AIC F value    Pr(>F)    
+ x3    1    2028.9  2074.3 129.83 25.4307 3.001e-05 ***
+ x5    1    1473.9  2629.4 136.71 14.5743 0.0007509 ***
<none>               4103.2 147.62                      
+ x1    1     195.8  3907.4 148.20  1.3029 0.2640916    
+ x2    1       0.1  4103.1 149.61  0.0007 0.9786266    
- x4    1   10578.7 14681.9 182.59 69.6094 5.935e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Step:  AIC=129

In [11]:
summary(lm.step)


Call:
lm(formula = y ~ x4 + x3 + x2 + x1 + x5, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-13.7049  -2.8200   0.6207   3.9062  17.3937 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 326.37699   95.85338   3.405  0.00243 ** 
x4          -22.97522    2.69281  -8.532  1.4e-08 ***
x3            3.77078    1.45520   2.591  0.01633 *  
x2            2.54722    1.24523   2.046  0.05240 .  
x1            0.06780    0.02891   2.345  0.02803 *  
x5            2.45572    1.79259   1.370  0.18394    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.025 on 23 degrees of freedom
Multiple R-squared:  0.8991,	Adjusted R-squared:  0.8772 
F-statistic:    41 on 5 and 23 DF,  p-value: 1.034e-10


所有子集回归

In [36]:
library(leaps)
eps<- regsubsets(y~.,data=df,really.big=T,method="seqrep",nbest=9,nvmax=5)
expres<- summary(eps)

In [37]:
res<- data.frame(expres$outmat,rss=expres$rss)
res

,x1,x2,x3,x4,x5,rss
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1 ( 1 ),,,,*,,4103.244
1 ( 2 ),*,,,,,8898.148
1 ( 3 ),,,,,*,12897.564
1 ( 4 ),,,*,,,14497.458
1 ( 5 ),,*,,,,14528.121
2 ( 1 ),,,*,*,,2074.330
2 ( 2 ),,,,*,*,2629.359
2 ( 3 ),*,,,*,,3907.439
2 ( 4 ),,*,,*,,4103.128


In [14]:
names(expres)

[1] "which"  "rsq"    "rss"    "adjr2"  "cp"     "bic"    "outmat" "obj"

In [42]:
library(olsrr)
lm.all<-lm(y~.,data=df)
(df.all=ols_step_all_possible(lm.all))
N=29

,mindex,n,predictors,rsquare,adjr,predrsq,cp,aic,sbic,sbc,msep,fpe,apc,hsp
,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,1,x4,0.72052420,0.71017324,0.66925799,38.721380,231.9133,146.8415,236.0152,4407.994,162.45282,0.3208796,5.845077
1,2,1,x1,0.39393873,0.37149202,0.26286550,113.183914,254.3612,167.6536,258.4631,9559.019,352.28939,0.6958481,12.675425
5,3,1,x5,0.12153476,0.08899901,-0.07689869,175.292894,265.1259,177.9438,269.2278,13855.473,510.63151,1.0086082,18.372598
3,4,1,x3,0.01256447,-0.02400721,-0.10822289,200.138469,268.5170,181.2158,272.6189,15574.192,573.97342,1.1337223,20.651649
2,5,1,x2,0.01047594,-0.02617310,-0.10035893,200.614661,268.5783,181.2751,272.6801,15607.133,575.18743,1.1361202,20.695329
13,6,2,x3 x4,0.85871542,0.84784738,0.81361382,9.213338,214.1313,131.2144,219.6005,2317.527,88.03523,0.1738887,3.191277
15,7,2,x4 x5,0.82091186,0.80713584,0.74337520,17.832672,221.0072,136.8021,226.4764,2937.629,111.59085,0.2204162,4.045168
8,8,2,x1 x4,0.73386067,0.71338841,0.66912580,37.680621,232.4953,146.5192,237.9645,4365.552,165.83294,0.3275561,6.011444
11,9,2,x2 x4,0.72053206,0.69903453,0.63546717,40.719587,233.9125,147.7510,239.3817,4584.185,174.13807,0.3439605,6.312505


In [43]:
sst=sum((df$y-mean(df$y))^2)

In [49]:
rms=(sst*(1-df.all$rsquare))/(N-df.all$n)

In [47]:
N-df.all$n

[1] 28 28 28 28 28 27 27 27 27 27 27 27 27 27 27 26 26 26 26 26 26 26 26 26 26
[26] 25 25 25 25 25 24

In [51]:
(out.all=data.frame(predictors=df.all$predictors,rms=rms,aic=df.all$aic))

predictors,rms,aic
<chr>,<dbl>,<dbl>
x4,146.54442,231.9133
x1,317.79100,254.3612
x5,460.62727,265.1259
x3,517.76634,268.5170
x2,518.86147,268.5783
x3 x4,76.82704,214.1313
x4 x5,97.38368,221.0072
x1 x4,144.71995,232.4953
x2 x4,151.96771,233.9125
